In [ ]:
%pip install langgraph langchain mlflow cloudpickle pandas

In [ ]:
import sys
if 'agent_model' in sys.modules:
    del sys.modules['agent_model']
if 'tools' in sys.modules:
    del sys.modules['tools']

from agent_model import AGENT 

result = AGENT.predict({
    "input": [{
        "role": "user", 
        "content": "List all you tools "
    }]
})
print("\nRESULT:")
print(result.model_dump(exclude_none=True))

In [ ]:
import mlflow
import langgraph
import cloudpickle
import langchain
import databricks_langchain
from mlflow.types.responses import RESPONSES_AGENT_INPUT_EXAMPLE  

mlflow.set_registry_uri("databricks-uc")

pip_requirements = [
    f"mlflow>={mlflow.__version__}",
    f"cloudpickle=={cloudpickle.__version__}",
    "langgraph",
    f"langchain=={langchain.__version__}",
    "databricks_langchain",
    "pandas",
    "requests", 
    "python-dotenv"
]

catalog="YOUR DATABRICKS CATALOG"
schema="YOUR DATABRICKS SCHEMA"
name="YOUR DESIRED NAME FOR THE MODEL"
model_name = f"{catalog}.{schema}.{name}"

print(f"Logging and registering model to UC: '{model_name}'...")


with mlflow.start_run(run_name="Databricks LLM Agent (Code)") as run:
    
    logged_model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model="agent_model.py",
        code_paths=["tools.py"],
        pip_requirements=pip_requirements,
        input_example=RESPONSES_AGENT_INPUT_EXAMPLE, 
    )
    
    model_version = mlflow.register_model(
        model_uri=logged_model_info.model_uri,
        name=model_name 
    )

print(f"Model '{model_name}' (Version {model_version.version}) has been logged to Unity Catalog!")